In [10]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload
%autoreload 2
import numpy as np
from sktime_neuro.utils.pickle_wrapper import deserialise_if_exists, serialise_if_doesnt_exist
from sktime_neuro.datasets.matching_pennies_util import *
from sktime_neuro.transformations.multivariate_detrender import ColumnDetrender
from sktime_neuro.transformations.series_to_panel.eeg_epoching import epoch
from sktime_neuro.utils.mne_processing import create_annotation
from sktime.classification.kernel_based import RocketClassifier

from sklearn.metrics import classification_report,accuracy_score

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def detrend(data, n_jobs=4):
    dt = ColumnDetrender(data)
    return dt.detrend(n_jobs=n_jobs)

In [3]:
def process(participantNumber):
    raw = get_raw(participantNumber)
    annots = create_annotation(raw)
    data = raw.get_data().transpose()
    detrended = detrend(data, -1)
    sf = raw.info['sfreq']

    return detrended, annots, sf

In [4]:
p8, p8a, p8s = process(8)
p7, p7a, p7s = process(7)

/home/patchouli/programming/foss/sktime-neuro/sktime_neuro/datasets/matching_pennies_util.py:8: RuntimeWarning: Did not find any eeg.json associated with sub-08_task-matchingpennies.

The search_str was "/home/patchouli/programming/foss/sktime-neuro/cache/eeg_matchingpennies/sub-08/**/eeg/sub-08*eeg.json"
  return read_raw_bids(bids_path=bidspath, verbose=False)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of  10 | elapsed:   25.6s remaining:   59.7s
[Parallel(n_jobs=-1)]: Done   5 out of  10 | elapsed:   26.6s remaining:   26.6s
[Parallel(n_jobs=-1)]: Done   7 out of  10 | elapsed:   27.5s remaining:   11.8s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   29.2s finished
/home/patchouli/programming/foss/sktime-neuro/sktime_neuro/datasets/matching_pennies_util.py:8: RuntimeWarning: Did not find any eeg.json associated with sub-07_task-matchingpennies.

The search_str was "/home/patchouli/programming/foss/sktime-

In [5]:
onsetlabels = ["raised-right/match-true", "raised-right/match-false",
          "raised-left/match-true", "raised-left/match-false"]
#epoched, labels = epoch(p8d, annotation, onsetlabels, [-2,2], sfreq=p8r.info['sfreq'])
p8e, p8l = epoch(p8, p8a, onsetlabels, [-2,2], sfreq=p8s)
p7e, p7l = epoch(p7, p7a, onsetlabels, [-2,2], sfreq=p7s)

In [6]:
#classcount = p8e.shape[1]
rocket = RocketClassifier(n_jobs=-1)
rocket.fit(p8e, p8l)
#pred = rocket.predict(p7e)

RocketClassifier(n_jobs=-1)

In [13]:
ypred = rocket.predict(p7e)

In [18]:
print(classification_report(ypred, p7l))

                          precision    recall  f1-score   support

 raised-left/match-false       0.72      0.23      0.35       158
  raised-left/match-true       0.04      0.44      0.07         9
raised-right/match-false       0.08      0.24      0.12        17
 raised-right/match-true       0.55      0.48      0.52       116

                accuracy                           0.33       300
               macro avg       0.35      0.35      0.26       300
            weighted avg       0.60      0.33      0.39       300

